# 조회에 필요한 데이터 불러오기

In [2]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [18]:
files = sorted(glob('./data/*.csv'))
date_cols = ['date']
data_all = pd.DataFrame()
for file in tqdm(files):
    df_day = pd.read_csv(file, parse_dates=date_cols, usecols=['press', 'title', 'date', 'link']) # date 컬럼 datetime 형식으로 불러오기
    data_all = pd.concat([data_all, df_day])
data_all.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [00:50<00:00,  7.27it/s]


In [3]:
data_all

,press,title,date,link
0,중앙일보,프로배구 2·3일 경기 연기.. 중계방송 스태프 코로나19 확진,2021-01-01,https://v.daum.net/v/20210101233343622
1,세계일보,"""산에서 커피 팔고 싶어"" 조두순의 소박한 꿈 이뤄질까? 시민들 '불안'",2021-01-01,https://v.daum.net/v/20210101233139613
2,서울신문,확진 1000명 목전 두고..秋 '구치소 집단감염' 첫 사과(종합),2021-01-01,https://v.daum.net/v/20210101233101608
3,국민일보,문건 유출? 미루기?.. 벌써 다 퍼진 '2.5단계 연장안',2021-01-01,https://v.daum.net/v/20210101231448533
4,뉴스1,'해변 폐쇄·드론 감시' 강릉 해맞이객 90% 줄어..그래도 1만6천명,2021-01-01,https://v.daum.net/v/20210101231238518
...,...,...,...,...
1411408,세계일보,"한국남동발전, 8년째 취약계층 평생학습 지원",2022-12-31,https://v.daum.net/v/20221231010122552
1411409,세계일보,"KIS제주, 하버드대 2년 연속 합격생 배출",2022-12-31,https://v.daum.net/v/20221231010117551
1411410,한국일보,늘어나는 방음터널... 번지면 대형 화재인데 방염 규정은 전무,2022-12-31,https://v.daum.net/v/20221231001003411
1411411,국민일보,[핫인픽] #배달할증료 #한파·폭설 출근길 #메시 지폐,2022-12-31,https://v.daum.net/v/20221231000423387


# 전체 데이터 유사도 추천

## 모델 불러오기

In [4]:
from sentence_transformers import SentenceTransformer
import torch
import pickle

In [5]:
model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')

In [7]:
with open("./model/pickled_embeddings_ddobokki_klue-roberta-small-nli-sts.bin", "rb") as f:
    embeddings = torch.load(f, map_location='cpu')

## 형태소 추출

In [8]:
def string_tokenizer(content):
    import pandas as pd
    from konlpy.tag import Okt
    okt = Okt()
    
    content = pd.Series(content)
    # 한글, 숫자, 공백만 남기기
    content = content.str.replace('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)
    token_content = okt.pos(content[0], norm=True, stem=True)
    return token_content

In [9]:
except_list = ['Exclamation', 'Josa', 'KoreanParticle', 'Determiner',  'Eomi', 'Suffix',  'VerbPrefix', 'PreEomi']
include_list = ['Verb',  'Noun']

In [10]:
# 230207 기현
def tag_except_list(except_list, token_content):
    filtered_list= []
    for tag in token_content :
        if tag[1] in except_list:
            continue
        filtered_list.append(tag[0])
    content = " ".join(filtered_list)
    return content

## 다음 url로 유사 기사 추천

In [11]:
stop_content = ["무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
rep_list = ['기사내용 요약']

In [12]:
def create_soup(url):
    from bs4 import BeautifulSoup
    import requests
    i = 0
    ## 요청 오류시 10번  재시도
    while i < 10 :
        try:
            res = requests.get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
            break
        except:
            i += 1
            
    return soup

In [13]:
def content_only_scraper(link):
    article_soup = create_soup(link)

    # 본문 정리
    article = article_soup.find_all('section')[1]
    content = article.find_all(True, attrs={'dmcf-ptype':'general'})
    rst = []
    for para in content:
        for tmp in para.text.split('\n'):
            if tmp.strip() != '':
                rst.append(tmp.strip())
    content = []
    for c in rst:
        for i in stop_content:
            if i in c:
                break
        else:
            for rep in rep_list:
                c = c.replace(rep, '')
            content.append(c)
    content = ' '.join(content)
    if len(content) < 200:
        return False
    
    return content

In [32]:
def get_recommend_by_url(url, n, drange, press):
    import torch, time
    from sentence_transformers import util
    # 다음 뉴스인지 확인
    if not url.startswith('https://v.daum.net/v/'):
        return False

    ## 입력 처리
    # 본문 스크랩
    content = content_only_scraper(url)
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))

    # sbert 연산
    start = time.time()
    embeded_content = model.encode([token], convert_to_tensor=True)
    print(f"입력기사 임베딩 시간 : {time.time()-start}")
    ## 조회 한정
    # 기간 & 언론사
    start = time.time()
    data_filtered_idx = data_all[(data_all['press']==press) & (data_all['date'].isin(pd.date_range(drange[0], drange[1])))].index
    embeddings_filtered = embeddings[data_filtered_idx]
    cos_sim_res = util.cos_sim(embeded_content, embeddings_filtered)
    print(f"코사인 계산 시간 : {time.time()-start}")
    
    # 유사도 정렬, 추출
    sim_scores_n = torch.topk(cos_sim_res, k=n)
    
    # 해당하는 기사 가져오기
    article_idx = data_filtered_idx[sim_scores_n[1][0]]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim.item(), 4)*100 for sim in sim_scores_n[0][0]]
     
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [33]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2022-01-01", "2022-12-31"), "뉴시스")

입력기사 임베딩 시간 : 0.30916643142700195
코사인 계산 시간 : 0.4333994388580322
CPU times: total: 3.36 s
Wall time: 1.33 s


,press,similarity,title,date,link
414971,뉴시스,85.95,"정경심, 내일 형집행정지 기로..중앙지검서 심의위 개최",2022-08-17,https://v.daum.net/v/20220817100516164
614399,뉴시스,85.74,"조국 징역 5년 구형…검찰 ""명백한 사실조차 인정 안해""(종합)",2022-12-02,https://v.daum.net/v/20221202145411164
586600,뉴시스,85.68,"정경심 ""딸·아들·남편 삶 망쳐…천하의 나쁜 아내·엄마"" 법정 오열",2022-11-18,https://v.daum.net/v/20221118172209373
283648,뉴시스,85.59,"'3선 교육감' 조희연, 해직교사 특채 의혹에 발목 잡히나",2022-06-03,https://v.daum.net/v/20220603070016675
587406,뉴시스,85.36,"檢 ""입시비리 중대범죄"", 정경심 측 ""부당 기소""…징역 2년 구형",2022-11-18,https://v.daum.net/v/20221118115238952
52264,뉴시스,84.82,"정경심 변호인 ""안타깝다는 말씀 밖에 못 드리겠다..불쌍하다""",2022-01-27,https://v.daum.net/v/20220127105626847
51345,뉴시스,84.62,조국 재판 남았다..'동일 증거·동일 혐의' 불리한 영향 불가피,2022-01-27,https://v.daum.net/v/20220127145128925
587541,뉴시스,84.33,"檢, 정경심 '아들 입시비리 혐의' 징역 2년 구형…""중대범죄""",2022-11-18,https://v.daum.net/v/20221118110927242
385867,뉴시스,84.20,"정경심, 형집행정지 신청..""허리디스크 파열""",2022-08-01,https://v.daum.net/v/20220801174105172
389088,뉴시스,83.92,"[자녀살해②]반복되는 비극..""생활고·스트레스·돌봄부담에..""",2022-08-02,https://v.daum.net/v/20220802122026918
